## Example Workbook For Creating Propeller CHARM Input Files from OpenVSP
This workbook is designed to be a simple example of how to generate input files for CHARM from an OpenVSP file using Uber developed python helper packages

### Import packages
The first step in using the utilties is to import the distributed packages as well as any other packages needed to manipulate the case setup

In [ ]:
# Import the helper packages
import charm.input_automation as charm
import openvsp as vsp
import utilities.units as u

# Import standard useful python packages
import os
import math

### Define Case Specific Constants
Next we will define some constants for the specifics of this example case

In [ ]:
# directory definitions
base_directory = "simple_prop"
template_directory = "templates"
geom_directory = "geom"
run_directory = "runs"

overwrite_files = True

# vsp specifics
vsp_filename = "prop.vsp3"
vsp_propname = "Prop"
vsp_charm_set_name = "Charm"

# charm template files
charm_rw_template = "proprw.inp" # rotor wake template file
charm_rc_template = "prop.inp" # run characteristics file
charm_casename = "prop" # name of the charm case

# Solution Specifics
airspeed_fts = 100.0 * u.mph2fts # Airspeed in feet per second
tilt_deg = 0.0 # 0.0 = axial flow, 90.0 = edgewise
alpha_deg = 0.0 # 0.0 air along x axis, 90.0 vertical climb
prop_rpm = 1500.0 # propeller RPM
prop_collective = 0.0 # propeller collective

### Import OpenVSP Geometry & Plot

In [ ]:
# Reset VSP's state
vsp.VSPRenew()
vsp.ClearVSPModel()
vsp.DeleteAllResults()

# Load in the vsp model
vsp.ReadVSPFile(os.path.join(base_directory, geom_directory, vsp_filename))

# Get the geom id of the propeller
vsp_propid = vsp.FindGeom(vsp_propname, 0)

# Apply the tilt angle in vsp
vsp.SetParmVal(vsp_propid, "Y_Rel_Rotation", "XForm", tilt_deg)

# Update the vsp model
vsp.Update()

# Run degen geom and plot
vsp_charm_set_index = vsp.GetSetIndex(vsp_charm_set_name)
dg_mgr = vsp.run_degen_geom(set_name=vsp_charm_set_name)
prop_info = vsp.get_propeller_thrust_vectors(prop_set=vsp_charm_set_index)

# Plot propellers
prop_fig = vsp.plot_propeller_info(prop_info, vector_scale=20)

### Create inputs for charm
Now that the geometry has been loaded, and the we have verified its position, we can now generate the charm input files

In [ ]:
# Clear out old vsp results
vsp.DeleteAllResults()

# Run degen geom on the charm set
degen_mgr = vsp.run_degen_geom(set_index=vsp_charm_set_index)

# Read in template file
rotor_rw = []
with open(os.path.join(base_directory, template_directory, charm_rw_template), "r") as f:
    rotor_rw = f.readlines()


# Initialize the rotor settings object
rotor_settings = charm.build_default_rotor_settings(degen_mgr=degen_mgr, default_rpm=prop_rpm,
                                                    default_template=rotor_rw)
rotor_settings.icoll = 0
rotor_settings.initial_collective = prop_collective
rotor_settings.ct = 0.01
rotor_settings.nspan_override = 50

# Create velocity vector
u_speed = airspeed_fts*math.cos(alpha_deg * u.deg2rad)
v = 0.0
w = airspeed_fts*math.sin(alpha_deg * u.deg2rad)

# Create input file list
files_to_write = charm.build_charm_input_files(degen_mgr=degen_mgr, case_name=charm_casename,
                                              rotor_settings=rotor_settings,
                                              unit_factor=u.in2ft,
                                              run_char_filename=os.path.join(base_directory,
                                                                             template_directory,charm_rc_template),
                                              velocity=[u_speed, v, w])


run_dir_name = 'run_v{:.3}_tilt_{:3.2f}_alpha_{:3.2f}'.format(airspeed_fts, tilt_deg, alpha_deg)

### Write out the files
Now that the files have been generate and are currently stored in memory, we can write them out a directory where we can execute charm

In [ ]:
cwd = os.getcwd()
try:
    charm_exe_dir = os.path.join(base_directory, run_directory, run_dir_name)
    os.makedirs(charm_exe_dir, exist_ok=overwrite_files)
    os.chdir(charm_exe_dir)
    
    for filename, file_contents in files_to_write.items():
        with open(filename, 'w') as f:
            f.write(file_contents)
finally:
    os.chdir(cwd)

### Next Steps
Now that the charm input files have been generated, you can go to the output directory and start running CHARM.